<a href="https://colab.research.google.com/github/yhatpub/yhatpub/blob/notebook/notebooks/fastai/lesson6_multicat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fastai Lesson 6 Multicat on YHat.pub

This notebook picks up from [Fastai Fastbook 6 multicat](https://github.com/fastai/fastbook/blob/master/06_multicat.ipynb) X to [YHat.pub](https://yhat.pub)

### Installs
The following cell installs pytorch, fastai and yhat_params, which is used to decorate your `predict` function.

In [1]:
!pip install -q --upgrade --no-cache-dir torch torchvision torchaudio
!pip install -q --upgrade --no-cache-dir fastai

!pip install -q --no-cache-dir git+https://github.com/yhatpub/yhat_params.git@main

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done


### Imports
**Warning** don't place `pip installs` and `imports` in the same cell. The imports might not work correctly if done that way.

In [2]:
from fastai.vision.all import *
from yhat_params.yhat_tools import FieldType, inference_predict

### Download Model
Google drive does not allow direct downloads for files over 100MB, so you'll need to follow the snippet below to get the download url.

In [3]:
#file copied from google drive
google_drive_url = "https://drive.google.com/file/d/1eJ4SBHbxpbR6aIrx1J4Lu4bFTyL9Huj1/view?usp=sharing"
import os
os.environ['GOOGLE_FILE_ID'] = google_drive_url.split('/')[5]
os.environ['GDRIVE_URL'] = f'https://docs.google.com/uc?export=download&id={os.environ["GOOGLE_FILE_ID"]}'
!echo "This is the Google drive download url $GDRIVE_URL"

This is the Google drive download url https://docs.google.com/uc?export=download&id=1eJ4SBHbxpbR6aIrx1J4Lu4bFTyL9Huj1


`wget` it from google drive. This script places the model in a `model` folder

In [4]:
!wget -q --no-check-certificate $GDRIVE_URL -r -A 'uc*' -e robots=off -nd
!mkdir -p models
!mv $(ls -S uc* | head -1) ./models/export.pth

Now let's do the same for the labels csv

In [ ]:
#cleanup from previous download
!rm uc*
#file copied from google drive
google_drive_url = "https://drive.google.com/file/d/1p6gRb0v8jaBiDSGRKsYPpdnEi4IaJrcw/view?usp=sharing"
import os
os.environ['GOOGLE_FILE_ID'] = google_drive_url.split('/')[5]
os.environ['GDRIVE_URL'] = f'https://docs.google.com/uc?export=download&id={os.environ["GOOGLE_FILE_ID"]}'
!echo "This is the Google drive download url $GDRIVE_URL"

In [ ]:
!wget -q --no-check-certificate $GDRIVE_URL -r -A 'uc*' -e robots=off -nd
!mkdir -p models
!mv $(ls -S uc* | head -1) ./models/vocab.csv

verify the model exists. **Warning** YHat is pretty finicky about where you place your models. Make sure you create a `model` directory and download your model(s) there  

In [ ]:
!ls -l ./models

**Warning** if your model depends on extra functions, as lesson6 does, you'll need to include those functions. In python, functions are not included in the exported `pkl` file.

In [ ]:
!less models/vocab.csv

In [8]:
import csv
from PIL import Image
import os

with open("models/vocab.csv") as file_name:
    file_read = csv.reader(file_name)
    labels = list(file_read)

new_image = Image.new(mode="RGB", size=(1,1))
if not os.path.exists('data'):
    os.makedirs('data')
new_image.save('data/dummyimage.jpg')

dblock = DataBlock(blocks=(ImageBlock, CategoryBlock),
                   get_x=ColReader('images'), 
                   get_y=ColReader('labels'),
                   item_tfms=Resize(192),
                   batch_tfms=Normalize.from_stats(*imagenet_stats))

df = pd.DataFrame(
     {
        'images': [
                  'data/dummyimage.jpg', 
                  ]*20, 
        'labels': labels*2, 
        'valid': [True] *20
     },
    )
dls = dblock.dataloaders(df, bs=64, num_workers=workers)

ValueError: ignored

In [ ]:
!less models/vocab.csv

0
aeroplane
bicycle
bird
boat
bottle
bus
car
cat
chair
cow
diningtable
dog
horse
motorbike
person
pottedplant
sheep
sofa
train
tvmonitor
models/vocab.csv (END)

### Load your learner
The following is the equivalent of torch `torch.load` or ts `model.load_weights`

In [ ]:
learn_inf = cnn_learner(dls, resnet50, metrics=partial(accuracy_multi, thresh=0.2))

In [ ]:
learn_inf.load('')

In [ ]:
!ls -ls models/

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
    res = torch.load('models/export.pth', map_location='cpu' , pickle_module=pickle)
    if hasattr(res, 'to_fp32'): 
        res = res.to_fp32()
        print("!")
    if True: 
        res.dls.cpu()
        print("##")

And write your predict function. Note, you will need to decorate your function with <a href="https://github.com/yhatpub/yhat_params">inference_predict</a> which takes 2 parameters, a `dic` for input and output.

**Info** These parameters are how YHat.pub maps your predict functions input/output of the web interface. The `dic` key is how you access the variable and the value is it's type. You can use autocomplete to see all the input/output types and more documentation on `inference_predict` is available at the link. 

In [ ]:
input = {"image": FieldType.PIL}
output = {"text": FieldType.Text}

@inference_predict(input=input, output=output)
def predict(params):
    img = PILImage.create(np.array(params["image"].convert("RGB")))
    result = learn_inf.predict(img)
    return {"text": str(result[0])}

### Test
First, import `in_colab` since you only want to run this test in colab. YHat will use this colab in a callable API, so you don't want your test to run every time `predict` is called. Next, import `inference_test` which is a function to make sure your `predict` will run with YHat.

Now, inside a `in_colab` boolean, first get whatever test data you'll need, in this case, an image. Then you'll call your predict function, wrapped inside  `inference_test`, passing in the same params you defined above. If something is missing, you should see an informative error. Otherwise, you'll see something like
`Please take a look and verify the results`

In [ ]:
from yhat_params.yhat_tools import in_colab, inference_test

if in_colab():
    import urllib.request
    from PIL import Image
    urllib.request.urlretrieve("https://s3.amazonaws.com/cdn-origin-etr.akc.org/wp-content/uploads/2017/11/11234019/Bulldog-standing-in-the-grass.jpg", "input_image.jpg")
    img = Image.open("input_image.jpg")
    inference_test(predict_func=predict, params={'image': img})

### That's it

If you run into errors, feel free to hop into Discord.

Otherwise, you'll now want to clear your outputs and save a public repo on Github